---
# **Instalación del entorno de Kaggle**



In [1]:
!pip install -q kaggle

---
# **Instalación de librerias**

In [2]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.2 MB/s eta 0:00:00


In [72]:
import pandas as pd
import numpy as np
import statistics
import unidecode
from google.colab import files
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

---
# **Lectura de los datos desde Kaggle**

In [4]:
!mkdir -p ~/.kaggle

In [5]:
!echo '{"username":"nicolsniovalderrama","key":"f7f9fda0ddecb368ca50593318da1412"}' > ~/.kaggle/kaggle.json

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c udea-ai4eng-20241

 52% 11.0M/21.2M [00:00<00:00, 113MB/s]
100% 21.2M/21.2M [00:00<00:00, 155MB/s]


In [8]:
!unzip udea-ai4eng-20241

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [9]:
df_train = pd.read_csv('train.csv')
df_train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


In [10]:
df_test = pd.read_csv('test.csv')

---
# **Tratamiento de datos**

## **Valores duplicados**

In [11]:
df_train.duplicated().sum()

0

### Eliminación de columnas repetidas

In [12]:
df0 = df_train.copy()
A = df0['FAMI_TIENEINTERNET'] == df0['FAMI_TIENEINTERNET.1']
print(f'Hay {len(A)} datos')
B = (A==False).sum()
print(f'De los cuales, {B} son diferentes entre las columnas FAMI_INTERNET Y FAMI_INTERNET.1')
C = B / len(A)
print(f'Los datos diferentes entre las dos columnas corresponde al {(C*100).round(2)}% deL total de los datos')

Hay 692500 datos
De los cuales, 26629 son diferentes entre las columnas FAMI_INTERNET Y FAMI_INTERNET.1
Los datos diferentes entre las dos columnas corresponde al 3.85% deL total de los datos


Debido a que las dos columnas difieren en solo 3.85%, se toma la desición de eliminar la columna 'FAMI_TIENEINTERNET.1'

In [13]:
df1 = df0.drop('FAMI_TIENEINTERNET.1', axis = 1)

In [14]:
df_test = df_test.drop('FAMI_TIENEINTERNET.1', axis = 1)

## **Información no relevante**

Se eliminan las variables PERIODO, ESTU_PRGM_DEPARTAMENTO, ESTU_PRIVADO_LIBERTAD, FAMI_TIENELAVADORA y FAMI_TIENEAUTOMOVIL ya que se considera que no aportaran al modelo.

In [15]:
df1 = df1.drop(columns = ['PERIODO', 'ESTU_PRGM_DEPARTAMENTO',
                          'ESTU_PRIVADO_LIBERTAD', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL'], axis = 1)

In [16]:
df_test = df_test.drop(columns = ['PERIODO', 'ESTU_PRGM_DEPARTAMENTO',
                          'ESTU_PRIVADO_LIBERTAD', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL'], axis = 1)

## **Conversión de variables categoricas en numericas**

### **Transformación de ESTU_VALORMATRICULAUNIVERSIDAD**

In [17]:
df1['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()

array(['Entre 5.5 millones y menos de 7 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones', 'Más de 7 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'No pagó matrícula', nan], dtype=object)

In [18]:
df1['ESTU_VALORMATRICULAUNIVERSIDAD'] = df1['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({'No pagó matrícula': 0,
                                                                                       'Menos de 500 mil': 250000,
                                                                                       'Entre 500 mil y menos de 1 millón': 750000,
                                                                                       'Entre 1 millón y menos de 2.5 millones': 1750000,
                                                                                       'Entre 2.5 millones y menos de 4 millones': 3250000,
                                                                                       'Entre 4 millones y menos de 5.5 millones': 4750000,
                                                                                       'Entre 5.5 millones y menos de 7 millones': 6250000,
                                                                                       'Más de 7 millones': 7000000
                                                                                       })

In [19]:
df_test['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_test['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({'No pagó matrícula': 0,
                                                                                       'Menos de 500 mil': 250000,
                                                                                       'Entre 500 mil y menos de 1 millón': 750000,
                                                                                       'Entre 1 millón y menos de 2.5 millones': 1750000,
                                                                                       'Entre 2.5 millones y menos de 4 millones': 3250000,
                                                                                       'Entre 4 millones y menos de 5.5 millones': 4750000,
                                                                                       'Entre 5.5 millones y menos de 7 millones': 6250000,
                                                                                       'Más de 7 millones': 7000000
                                                                                       })

### **Transformación de FAMI_ESTRATOVIVIENDA**

In [20]:
df1['FAMI_ESTRATOVIVIENDA'].unique()

array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
       nan, 'Estrato 6', 'Sin Estrato'], dtype=object)

In [24]:
df1['FAMI_ESTRATOVIVIENDA'] = df1['FAMI_ESTRATOVIVIENDA'].replace('Estrato ', '', regex=True)
df1['FAMI_ESTRATOVIVIENDA'].unique()

In [25]:
# Reemplazar 'Sin Estrato' por NaN en la columna 'FAMI_ESTRATOVIVIENDA'
df1['FAMI_ESTRATOVIVIENDA'].replace('Sin Estrato', np.nan, inplace=True)

In [26]:
df_test['FAMI_ESTRATOVIVIENDA'] = df1['FAMI_ESTRATOVIVIENDA'].replace('Estrato ', '', regex=True)
df_test['FAMI_ESTRATOVIVIENDA'].unique()

array(['3', '4', '5', '2', '1', nan, '6'], dtype=object)

In [27]:
# Reemplazar 'Sin Estrato' por NaN en la columna 'FAMI_ESTRATOVIVIENDA'
df_test['FAMI_ESTRATOVIVIENDA'].replace('Sin Estrato', np.nan, inplace=True)

### **Tranformación de ESTU_HORASSEMANATRABAJA**

In [28]:
df1['ESTU_HORASSEMANATRABAJA'].unique()

array(['Menos de 10 horas', '0', 'Más de 30 horas', 'Entre 21 y 30 horas',
       'Entre 11 y 20 horas', nan], dtype=object)

In [29]:
df1['ESTU_HORASSEMANATRABAJA'] = df1['ESTU_HORASSEMANATRABAJA'].replace({'Menos de 10 horas': 5,
                                                                         'Entre 11 y 20 horas': 15.5,
                                                                         'Entre 21 y 30 horas': 25.5,
                                                                         'Más de 30 horas': 30})

In [30]:
df_test['ESTU_HORASSEMANATRABAJA'] = df_test['ESTU_HORASSEMANATRABAJA'].replace({'Menos de 10 horas': 5,
                                                                         'Entre 11 y 20 horas': 15.5,
                                                                         'Entre 21 y 30 horas': 25.5,
                                                                         'Más de 30 horas': 30})

### **Transformación de FAMI_EDUCACIONPADRE**

In [31]:
df1['FAMI_EDUCACIONPADRE'].unique()

array(['Técnica o tecnológica incompleta',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'No sabe',
       'Primaria completa', 'Educación profesional completa',
       'Educación profesional incompleta', 'Primaria incompleta',
       'Postgrado', nan, 'Secundaria (Bachillerato) incompleta',
       'Ninguno', 'No Aplica'], dtype=object)

In [32]:
df1['FAMI_EDUCACIONPADRE'] = df1['FAMI_EDUCACIONPADRE'].replace({'No sabe': 0, 'Ninguno': 0, 'No Aplica': 0,
                                                                 'Primaria incompleta': 1, 'Primaria completa': 2,
                                                                 'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
                                                                 'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
                                                                 'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
                                                                 'Postgrado': 9})

In [33]:
df_test['FAMI_EDUCACIONPADRE'] = df_test['FAMI_EDUCACIONPADRE'].replace({'No sabe': 0, 'Ninguno': 0, 'No Aplica': 0,
                                                                 'Primaria incompleta': 1, 'Primaria completa': 2,
                                                                 'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
                                                                 'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
                                                                 'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
                                                                 'Postgrado': 9})

### **Transformación de FAMI_EDUCACIONMADRE**

In [34]:
df1['FAMI_EDUCACIONMADRE'].unique()

array(['Postgrado', 'Técnica o tecnológica incompleta',
       'Secundaria (Bachillerato) completa', 'Primaria completa',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) incompleta',
       'Educación profesional incompleta',
       'Educación profesional completa', 'Primaria incompleta', nan,
       'Ninguno', 'No Aplica', 'No sabe'], dtype=object)

In [35]:
df1['FAMI_EDUCACIONMADRE'] = df1['FAMI_EDUCACIONMADRE'].replace({'No sabe': 0, 'Ninguno': 0, 'No Aplica': 0,
                                                                 'Primaria incompleta': 1, 'Primaria completa': 2,
                                                                 'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
                                                                 'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
                                                                 'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
                                                                 'Postgrado': 9})

In [36]:
df_test['FAMI_EDUCACIONMADRE'] = df_test['FAMI_EDUCACIONMADRE'].replace({'No sabe': 0, 'Ninguno': 0, 'No Aplica': 0,
                                                                 'Primaria incompleta': 1, 'Primaria completa': 2,
                                                                 'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
                                                                 'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
                                                                 'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
                                                                 'Postgrado': 9})

## **Codificación binaria**

In [37]:
var_yesno = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']

for var in var_yesno:
    df1.loc[df1[var] == 'Si', var] = 1
    df1.loc[df1[var] == 'No', var] = 0
    df_test.loc[df_test[var] == 'Si', var] = 1
    df_test.loc[df_test[var] == 'No', var] = 0

In [38]:
df1.head()

,ID,ESTU_PRGM_ACADEMICO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,ENFERMERIA,6250000.0,5,3,1,5.0,0,1,9.0,medio-alto
1,645256,DERECHO,3250000.0,0,3,0,6.0,0,1,5.0,bajo
2,308367,MERCADEO Y PUBLICIDAD,3250000.0,30,3,1,4.0,0,0,4.0,bajo
3,470353,ADMINISTRACION DE EMPRESAS,4750000.0,0,4,1,0.0,0,1,4.0,alto
4,989032,PSICOLOGIA,3250000.0,25.5,3,1,2.0,0,1,2.0,medio-bajo


## **Valores nulos**


In [39]:
df1.isnull().sum()

ID                                    0
ESTU_PRGM_ACADEMICO                   0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              35426
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_EDUCACIONMADRE               23664
RENDIMIENTO_GLOBAL                    0
dtype: int64

## **Imputación de valores**

In [40]:
def imputacion(df, strategy='most_frequent'):
    def columnas_na(df):
       # Encontrar las columnas con valores faltantes
        name_columns = df.columns[df.isnull().any()].tolist()
        return name_columns
    # Encontrar las columnas con valores faltantes
    name_columns = columnas_na(df)
    # Crear un imputador con la estrategia especificada
    imputer = SimpleImputer(strategy=strategy)
    # Imputar los valores faltantes en las columnas especificadas
    df_imputed = df.copy()
    df_imputed[name_columns] = imputer.fit_transform(df_imputed[name_columns])

    return df_imputed

In [41]:
df2 = imputacion(df1, strategy='most_frequent')
df_test = imputacion(df_test)

In [42]:
df2.isnull().sum()

ID                                0
ESTU_PRGM_ACADEMICO               0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
ESTU_PAGOMATRICULAPROPIO          0
FAMI_TIENECOMPUTADOR              0
FAMI_EDUCACIONMADRE               0
RENDIMIENTO_GLOBAL                0
dtype: int64

In [43]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   ESTU_PRGM_ACADEMICO             692500 non-null  object
 2   ESTU_VALORMATRICULAUNIVERSIDAD  692500 non-null  object
 3   ESTU_HORASSEMANATRABAJA         692500 non-null  object
 4   FAMI_ESTRATOVIVIENDA            692500 non-null  object
 5   FAMI_TIENEINTERNET              692500 non-null  object
 6   FAMI_EDUCACIONPADRE             692500 non-null  object
 7   ESTU_PAGOMATRICULAPROPIO        692500 non-null  object
 8   FAMI_TIENECOMPUTADOR            692500 non-null  object
 9   FAMI_EDUCACIONMADRE             692500 non-null  object
 10  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(1), object(10)
memory usage: 58.1+ MB


## **Estandarización de la variable ESTU_VALORMATRICULAUNIVERSIDAD**



In [44]:
# Crear un objeto MinMaxScaler
scaler = MinMaxScaler()
# Ajustar y transformar la columna 'ESTU_VALORMATRICULAUNIVERSIDAD' usando .loc
df2.loc[:, 'ESTU_VALORMATRICULAUNIVERSIDAD'] = scaler.fit_transform(df2[['ESTU_VALORMATRICULAUNIVERSIDAD']]).astype(float)
df_test.loc[:, 'ESTU_VALORMATRICULAUNIVERSIDAD'] = scaler.fit_transform(df_test[['ESTU_VALORMATRICULAUNIVERSIDAD']]).astype(float)

# Redondear los valores a tres decimales usando.apply
df2.loc[:, 'ESTU_VALORMATRICULAUNIVERSIDAD'] = df2['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(lambda x: round(x, 3))
df_test.loc[:, 'ESTU_VALORMATRICULAUNIVERSIDAD'] = df_test['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(lambda x: round(x, 3))

In [45]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   ESTU_PRGM_ACADEMICO             692500 non-null  object
 2   ESTU_VALORMATRICULAUNIVERSIDAD  692500 non-null  object
 3   ESTU_HORASSEMANATRABAJA         692500 non-null  object
 4   FAMI_ESTRATOVIVIENDA            692500 non-null  object
 5   FAMI_TIENEINTERNET              692500 non-null  object
 6   FAMI_EDUCACIONPADRE             692500 non-null  object
 7   ESTU_PAGOMATRICULAPROPIO        692500 non-null  object
 8   FAMI_TIENECOMPUTADOR            692500 non-null  object
 9   FAMI_EDUCACIONMADRE             692500 non-null  object
 10  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(1), object(10)
memory usage: 58.1+ MB


In [46]:
df2.head()

,ID,ESTU_PRGM_ACADEMICO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,ENFERMERIA,0.893,5,3,1,5.0,0,1,9.0,medio-alto
1,645256,DERECHO,0.464,0,3,0,6.0,0,1,5.0,bajo
2,308367,MERCADEO Y PUBLICIDAD,0.464,30,3,1,4.0,0,0,4.0,bajo
3,470353,ADMINISTRACION DE EMPRESAS,0.679,0,4,1,0.0,0,1,4.0,alto
4,989032,PSICOLOGIA,0.464,25.5,3,1,2.0,0,1,2.0,medio-bajo


## **Estandarización de la variable ESTU_HORASSEMANATRABAJA**


In [47]:
# Crear un objeto MinMaxScaler
scaler = MinMaxScaler()
# Ajustar y transformar la columna 'ESTU_VALORMATRICULAUNIVERSIDAD' usando .loc
df2.loc[:, 'ESTU_HORASSEMANATRABAJA'] = scaler.fit_transform(df2[['ESTU_HORASSEMANATRABAJA']])
df2['ESTU_HORASSEMANATRABAJA'] = df2['ESTU_HORASSEMANATRABAJA'].astype(float)
df2.loc[:, 'ESTU_HORASSEMANATRABAJA'] = df2['ESTU_HORASSEMANATRABAJA'].round(3)

In [48]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              692500 non-null  int64  
 1   ESTU_PRGM_ACADEMICO             692500 non-null  object 
 2   ESTU_VALORMATRICULAUNIVERSIDAD  692500 non-null  object 
 3   ESTU_HORASSEMANATRABAJA         692500 non-null  float64
 4   FAMI_ESTRATOVIVIENDA            692500 non-null  object 
 5   FAMI_TIENEINTERNET              692500 non-null  object 
 6   FAMI_EDUCACIONPADRE             692500 non-null  object 
 7   ESTU_PAGOMATRICULAPROPIO        692500 non-null  object 
 8   FAMI_TIENECOMPUTADOR            692500 non-null  object 
 9   FAMI_EDUCACIONMADRE             692500 non-null  object 
 10  RENDIMIENTO_GLOBAL              692500 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 58.1+ MB


In [49]:
df2.head()

,ID,ESTU_PRGM_ACADEMICO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,ENFERMERIA,0.893,0.167,3,1,5.0,0,1,9.0,medio-alto
1,645256,DERECHO,0.464,0.000,3,0,6.0,0,1,5.0,bajo
2,308367,MERCADEO Y PUBLICIDAD,0.464,1.000,3,1,4.0,0,0,4.0,bajo
3,470353,ADMINISTRACION DE EMPRESAS,0.679,0.000,4,1,0.0,0,1,4.0,alto
4,989032,PSICOLOGIA,0.464,0.850,3,1,2.0,0,1,2.0,medio-bajo


In [50]:
# Crear un objeto MinMaxScaler
scaler = MinMaxScaler()
# Ajustar y transformar la columna 'ESTU_VALORMATRICULAUNIVERSIDAD' usando .loc
df_test.loc[:, 'ESTU_HORASSEMANATRABAJA'] = scaler.fit_transform(df_test[['ESTU_HORASSEMANATRABAJA']])
df_test['ESTU_HORASSEMANATRABAJA'] = df_test['ESTU_HORASSEMANATRABAJA'].astype(float)
df_test.loc[:, 'ESTU_HORASSEMANATRABAJA'] = df_test['ESTU_HORASSEMANATRABAJA'].round(3)

## **One hot columna ESTU_PRGM_ACADEMICO**

In [51]:
df3 = df2.copy()

In [52]:
# Aplicar la función unidecode a la columna 'NOMBRE'
df3['ESTU_PRGM_ACADEMICO'] = df3['ESTU_PRGM_ACADEMICO'].apply(unidecode.unidecode)

In [53]:
# Aplicar la función unidecode a la columna 'NOMBRE'
df_test['ESTU_PRGM_ACADEMICO'] = df_test['ESTU_PRGM_ACADEMICO'].apply(unidecode.unidecode)

In [54]:
df3['ESTU_PRGM_ACADEMICO'].nunique()

787

In [55]:
def replace_PACAD(df):
    # Definir las palabras clave y sus respectivos reemplazos
    reemplazos = {
        'INGENI': 'INGENIERÍA',
        'ADMIN': 'ADMINISTRACIÓN',
        'LICENCIATURA': 'EDUCACIÓN',
        'LICENC': 'EDUCACIÓN',
        'EDUCACION': 'EDUCACIÓN',
        'QU': 'QUÍMICA',
        'VETERINARIA': 'VETERINARIA',
        'ZOOTECNIA': 'VETERINARIA',
        'FARMACIA': 'CIENCIAS BIOMÉDICAS',
        'BACTERIOLOGIA': 'CIENCIAS BIOMÉDICAS',
        'MICROBIOLOGIA': 'CIENCIAS BIOMÉDICAS',
        'CIENCIAS BIOMEDICAS': 'CIENCIAS BIOMÉDICAS',
        'MEDICINA': 'CIENCIAS DE LA SALUD',
        'ENFERMERIA': 'CIENCIAS DE LA SALUD',
        'ODONTOLOGIA': 'CIENCIAS DE LA SALUD',
        'OPTOMETRIA': 'CIENCIAS DE LA SALUD',
        'FONOAUDIOLOGIA': 'CIENCIAS DE LA SALUD',
        'TERAPIA OCUPACIONAL': 'CIENCIAS DE LA SALUD',
        'NUTRICION': 'CIENCIAS NUTRICIONALES',
        'FISIOTERAPIA': 'REHABILITACIÓN',
        'TERAPIA RESPIRATORIA': 'REHABILITACIÓN',
        'REHABILITACION': 'REHABILITACIÓN',
        'TERAPIA': 'REHABILITACIÓN',
        'SALUD': 'ÁREAS DE LA SALUD',
        'PSICOLO': 'ÁREAS DE LA SALUD',
        'CARDIORRESPIRATORIA': 'ÁREAS DE LA SALUD',
        'GERONTOLOGIA': 'ÁREAS DE LA SALUD',
        'MILITARES': 'MILICIA',
        'NAVALES': 'MILICIA',
        'MILICIA': 'MILICIA',
        'MUSICA': 'ARTES',
        'GRAFICO': 'ARTES',
        'AUDIOVISUAL': 'ARTES',
        'VISUAL': 'ARTES',
        'ARTE': 'ARTES',
        'CINE': 'ARTES',
        'MODA': 'ARTES',
        'ARTES PLASTICAS': 'ARTES',
        'DANZA': 'ARTES',
        'ARTES': 'ARTES',
        'MUSICAL': 'ARTES',
        'DIGITAL': 'ARTES',
        'INTERACTIVOS': 'ARTES',
        'TELEVISION': 'ARTES',
        'BANDA': 'ARTES',
        'DERECHO': 'CIENCIAS POLÍTICAS',
        'JURISPRUDENCIA': 'CIENCIAS POLÍTICAS',
        'POLITICA': 'CIENCIAS POLÍTICAS',
        'POLITICOS': 'CIENCIAS POLÍTICAS',
        'POLITICO': 'CIENCIAS POLÍTICAS',
        'GOBIERNO': 'CIENCIAS POLÍTICAS',
        'FINANZAS': 'CIENCIAS ECONÓMICAS',
        'INTERNACIONALES': 'CIENCIAS ECONÓMICAS',
        'NEGOCIOS': 'CIENCIAS ECONÓMICAS',
        'MERCADEO': 'CIENCIAS ECONÓMICAS',
        'ECONOMIA': 'CIENCIAS ECONÓMICAS',
        'MERCADOLOGIA': 'CIENCIAS ECONÓMICAS',
        'ECONOM': 'CIENCIAS ECONÓMICAS',
        'CONTADURIA': 'CIENCIAS ECONÓMICAS',
        'COMERCIO': 'CIENCIAS ECONÓMICAS',
        'DEPORTE': 'DEPORTE',
        'DEPORTIVA': 'DEPORTE',
        'ENTRENAMIENTO': 'DEPORTE',
        'RECREACION': 'DEPORTE',
        'LETRAS': 'IDIOMAS',
        'LENGUAS': 'IDIOMAS',
        'FILOLOGIA': 'IDIOMAS',
        'LENGUAJES': 'IDIOMAS',
        'LITERATURA': 'IDIOMAS',
        'TRADUCCION': 'IDIOMAS',
        'LITERARIOS': 'IDIOMAS',
        'LINGUISTICA': 'IDIOMAS',
        'LITERARIA': 'IDIOMAS',
        'GASTRONOMIA': 'GASTRONOMÍA',
        'GASTRONO': 'GASTRONOMÍA',
        'GEOLOGIA': 'ESTUDIOS GEOLÓGICOS',
        'GEOGRAFIA': 'ESTUDIOS GEOLÓGICOS',
        'GEOCIENCIAS': 'ESTUDIOS GEOLÓGICOS',
        'GEOLOG': 'ESTUDIOS GEOLÓGICOS',
        'MATEMATICA': 'CIENCIAS EXACTAS Y NATURALES',
        'ESTADISTICAS': 'CIENCIAS EXACTAS Y NATURALES',
        'OCEANOGRAFIA': 'CIENCIAS EXACTAS Y NATURALES',
        'ASTRONOMIA': 'CIENCIAS EXACTAS Y NATURALES',
        'BIOLOGIA': 'CIENCIAS EXACTAS Y NATURALES',
        'FISICA': 'CIENCIAS EXACTAS Y NATURALES',
        'ECOLOGIA': 'CIENCIAS EXACTAS Y NATURALES',
        'COMUNICACION': 'COMUNICACIÓN SOCIAL',
        'PERIODISMO': 'COMUNICACIÓN SOCIAL',
        'BIBLICAS': 'TEOLOGÍA',
        'BIBLICOS': 'TEOLOGÍA',
        'RELIGION': 'TEOLOGÍA',
        'TEOLOGIA': 'TEOLOGÍA',
        'SOCIAL': 'CIENCIAS SOCIALES Y HUMANAS',
        'SOCIOLOGIA': 'CIENCIAS SOCIALES Y HUMANAS',
        'ANTROPOLOGIA': 'CIENCIAS SOCIALES Y HUMANAS',
        'HISTORIA': 'CIENCIAS SOCIALES Y HUMANAS',
        'FILOSOFIA': 'FILOSOFÍA',
        'INFORMACION': 'TIC',
        'BIBLIOTECNOLOGIA': 'TIC',
        'ARCHIVISTICA': 'TIC',
        'REALIZACION': 'TIC',
        'COMPUTACION': 'TIC',
        'AGRONOMIA': 'CIENCIAS AGRARIAS',
        'AGROINDUSTRIAL': 'CIENCIAS AGRARIAS',
        'ACUICULTURA': 'CIENCIAS AGRARIAS',
        'AGROINDUSTRIA': 'CIENCIAS AGRARIAS',
        'MARKETING': 'PUBLICIDAD',
        'PUBLICIDAD': 'PUBLICIDAD',
        'HOTELERIA Y TURISMO': 'OTROS ESTUDIOS',
        'CONTRUCCION': 'OTROS ESTUDIOS',
        'CRIMINALISTICA': 'OTROS ESTUDIOS',
        'GESTION': 'OTROS ESTUDIOS',
        'CONSTRUCCIONES': 'OTROS ESTUDIOS',
        'DISENO': 'OTROS ESTUDIOS',
        'DISE': 'OTROS ESTUDIOS',
        'FOTOGRAFIA': 'OTROS ESTUDIOS',
        'LOGISTICA': 'OTROS ESTUDIOS',
        'MERCANTES': 'OTROS ESTUDIOS',
        'CULTURAL': 'OTROS ESTUDIOS',
        'BIOTECNOLOGIA': 'OTROS ESTUDIOS',
        'URBANISMO': 'OTROS ESTUDIOS',
        'CRIMINAL': 'OTROS ESTUDIOS',
        'DESARROLLO': 'OTROS ESTUDIOS',
        'CONSERVACION': 'OTROS ESTUDIOS',
        'ANIMACION': 'OTROS ESTUDIOS',
        'TURISMO': 'OTROS ESTUDIOS',
        'PEDAGOGIA': 'OTROS ESTUDIOS',
    }

    # Iterar sobre los reemplazos y aplicar las transformaciones
    for palabra_clave, reemplazo in reemplazos.items():
        df.loc[df['ESTU_PRGM_ACADEMICO'].str.contains(palabra_clave, case=False, na=False), 'ESTU_PRGM_ACADEMICO'] = reemplazo

    return df

In [56]:
df3 = replace_PACAD(df3)
df_test = replace_PACAD(df_test)

In [57]:
df3['ESTU_PRGM_ACADEMICO'].nunique()

28

In [58]:
df3['ESTU_PRGM_ACADEMICO'].unique()

array(['ÁREAS DE LA SALUD', 'CIENCIAS POLÍTICAS', 'CIENCIAS ECONÓMICAS',
       'ADMINISTRACIÓN', 'VETERINARIA', 'INGENIERÍA', 'OTROS ESTUDIOS',
       'EDUCACIÓN', 'CIENCIAS SOCIALES Y HUMANAS', 'ARTES', 'ESTADISTICA',
       'QUÍMICA', 'REHABILITACIÓN', 'CIENCIAS BIOMÉDICAS', 'DEPORTE',
       'IDIOMAS', 'CIENCIAS NUTRICIONALES', 'FILOSOFÍA',
       'CIENCIAS EXACTAS Y NATURALES', 'TIC', 'MILICIA',
       'CIENCIAS AGRARIAS', 'TEOLOGÍA', 'ESTUDIOS GEOLÓGICOS',
       'CONSTRUCCION', 'PUBLICIDAD', 'GASTRONOMÍA',
       'CIENCIAS AMBIENTALES'], dtype=object)

In [59]:
df3 = pd.get_dummies(df3, columns=['ESTU_PRGM_ACADEMICO'])

In [60]:
df3 = df3.replace(True, 1)
df3 = df3.replace(False, 0)

In [61]:
df3.head()

,ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,...,ESTU_PRGM_ACADEMICO_INGENIERÍA,ESTU_PRGM_ACADEMICO_MILICIA,ESTU_PRGM_ACADEMICO_OTROS ESTUDIOS,ESTU_PRGM_ACADEMICO_PUBLICIDAD,ESTU_PRGM_ACADEMICO_QUÍMICA,ESTU_PRGM_ACADEMICO_REHABILITACIÓN,ESTU_PRGM_ACADEMICO_TEOLOGÍA,ESTU_PRGM_ACADEMICO_TIC,ESTU_PRGM_ACADEMICO_VETERINARIA,ESTU_PRGM_ACADEMICO_ÁREAS DE LA SALUD
0,904256,0.893,0.167,3,1,5.0,0,1,9.0,medio-alto,...,0,0,0,0,0,0,0,0,0,1
1,645256,0.464,0.000,3,0,6.0,0,1,5.0,bajo,...,0,0,0,0,0,0,0,0,0,0
2,308367,0.464,1.000,3,1,4.0,0,0,4.0,bajo,...,0,0,0,0,0,0,0,0,0,0
3,470353,0.679,0.000,4,1,0.0,0,1,4.0,alto,...,0,0,0,0,0,0,0,0,0,0
4,989032,0.464,0.850,3,1,2.0,0,1,2.0,medio-bajo,...,0,0,0,0,0,0,0,0,0,1


In [62]:
df_test = pd.get_dummies(df_test, columns=['ESTU_PRGM_ACADEMICO'])

In [63]:
df_test = df_test.replace(True, 1)
df_test = df_test.replace(False, 0)

## **Conversión a variables numericas de las columnas**

In [64]:
df3['FAMI_ESTRATOVIVIENDA'].value_counts()

FAMI_ESTRATOVIVIENDA
2    268097
3    210685
1    111991
4     65514
5     23608
6     12605
Name: count, dtype: int64

In [65]:
df3['FAMI_ESTRATOVIVIENDA'] = df3['FAMI_ESTRATOVIVIENDA'].astype(int)
df3['ESTU_HORASSEMANATRABAJA'] = df3['ESTU_HORASSEMANATRABAJA'].astype(int)
df3['FAMI_TIENEINTERNET'] = df3['FAMI_TIENEINTERNET'].astype(int)
df3['FAMI_TIENECOMPUTADOR'] = df3['FAMI_TIENECOMPUTADOR'].astype(int)
df3['ESTU_PAGOMATRICULAPROPIO'] = df3['ESTU_PAGOMATRICULAPROPIO'].astype(int)

In [66]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 38 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   ID                                                692500 non-null  int64  
 1   ESTU_VALORMATRICULAUNIVERSIDAD                    692500 non-null  float64
 2   ESTU_HORASSEMANATRABAJA                           692500 non-null  int64  
 3   FAMI_ESTRATOVIVIENDA                              692500 non-null  int64  
 4   FAMI_TIENEINTERNET                                692500 non-null  int64  
 5   FAMI_EDUCACIONPADRE                               692500 non-null  float64
 6   ESTU_PAGOMATRICULAPROPIO                          692500 non-null  int64  
 7   FAMI_TIENECOMPUTADOR                              692500 non-null  int64  
 8   FAMI_EDUCACIONMADRE                               692500 non-null  float64
 9   REND

In [67]:
df3.head()

,ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,...,ESTU_PRGM_ACADEMICO_INGENIERÍA,ESTU_PRGM_ACADEMICO_MILICIA,ESTU_PRGM_ACADEMICO_OTROS ESTUDIOS,ESTU_PRGM_ACADEMICO_PUBLICIDAD,ESTU_PRGM_ACADEMICO_QUÍMICA,ESTU_PRGM_ACADEMICO_REHABILITACIÓN,ESTU_PRGM_ACADEMICO_TEOLOGÍA,ESTU_PRGM_ACADEMICO_TIC,ESTU_PRGM_ACADEMICO_VETERINARIA,ESTU_PRGM_ACADEMICO_ÁREAS DE LA SALUD
0,904256,0.893,0,3,1,5.0,0,1,9.0,medio-alto,...,0,0,0,0,0,0,0,0,0,1
1,645256,0.464,0,3,0,6.0,0,1,5.0,bajo,...,0,0,0,0,0,0,0,0,0,0
2,308367,0.464,1,3,1,4.0,0,0,4.0,bajo,...,0,0,0,0,0,0,0,0,0,0
3,470353,0.679,0,4,1,0.0,0,1,4.0,alto,...,0,0,0,0,0,0,0,0,0,0
4,989032,0.464,0,3,1,2.0,0,1,2.0,medio-bajo,...,0,0,0,0,0,0,0,0,0,1


In [68]:
df_test['FAMI_ESTRATOVIVIENDA'] = df_test['FAMI_ESTRATOVIVIENDA'].astype(int)
df_test['ESTU_HORASSEMANATRABAJA'] = df_test['ESTU_HORASSEMANATRABAJA'].astype(int)
df_test['FAMI_TIENEINTERNET'] = df_test['FAMI_TIENEINTERNET'].astype(int)
df_test['FAMI_TIENECOMPUTADOR'] = df_test['FAMI_TIENECOMPUTADOR'].astype(int)
df_test['ESTU_PAGOMATRICULAPROPIO'] = df_test['ESTU_PAGOMATRICULAPROPIO'].astype(int)

___
# **Random Forest**

In [73]:
# Random Forest
X = df3.drop("RENDIMIENTO_GLOBAL", axis=1)
y = df3["RENDIMIENTO_GLOBAL"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=10,
                                  min_samples_leaf=2, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.39612996389891697


In [74]:
preds_ts = rf_model.predict(df_test)

In [75]:
submission = pd.DataFrame([df_test.ID, pd.Series(preds_ts, name="RENDIMIENTO_GLOBAL")]).T
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-bajo
2,499179,alto
3,782980,bajo
4,785185,bajo


In [76]:
submission.shape

(296786, 2)

In [77]:
from google.colab import files
submission.to_csv("solución_kaggle.csv", index=False)
files.download("solución_kaggle.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>